In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import glob
import os
#pd.set_option('display.max_rows', 200)

In [12]:
data = pd.read_excel('..\\Datos Partidos\\Newells_Situaciones.xlsx')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 15 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Session Start Date   41 non-null     datetime64[ns]
 1   Event                41 non-null     object        
 2   Session Name         41 non-null     object        
 3   Session Start        41 non-null     object        
 4   Session End          41 non-null     object        
 5   Tag Description      41 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            41 non-null     object        
 8   Tag End              41 non-null     object        
 9   Tag Duration (secs)  41 non-null     float64       
 10  Categoria            41 non-null     object        
 11  Zona2                41 non-null     float64       
 12  Zona                 41 non-null     float64       
 13  Accion               41 non-null     

In [13]:
data.columns = ['Fecha','etiquetado','name','Session Start','Session End','Evento','Tag Notes','Tag Start','Tag End','Tag Duration (secs)','Categoria','Zona inicial','Zona Final','Accion','Jugada']

In [14]:
data.sample(5)

,Fecha,etiquetado,name,Session Start,Session End,Evento,Tag Notes,Tag Start,Tag End,Tag Duration (secs),Categoria,Zona inicial,Zona Final,Accion,Jugada
38,2022-06-25,Newells,Situaciones,23:21:12,01:36:02,A FAVOR,NaN,02:07:12,02:07:14,2000.0,ATAQUE DIRECTO,4.0,5.0,VOLVER,11.0
40,2022-06-25,Newells,Situaciones,23:21:12,01:36:02,A FAVOR,NaN,02:07:26,02:07:28,2000.0,ATAQUE DIRECTO,8.2,7.0,REMATE AFUERA,11.0
3,2022-06-25,Newells,Situaciones,23:21:12,01:36:02,A FAVOR,NaN,00:30:11,00:30:13,2000.0,TRANSICION,8.2,8.1,VOLVER,1.0
10,2022-06-25,Newells,Situaciones,23:21:12,01:36:02,En Contra,NaN,00:44:50,00:44:52,2000.0,PELOTA PARADA,5.0,2.1,VOLVER,3.0
2,2022-06-25,Newells,Situaciones,23:21:12,01:36:02,A FAVOR,NaN,00:30:08,00:30:10,2000.0,TRANSICION,8.1,5.0,VOLVER,1.0


In [15]:
'''CORREGIR MAYUSCULAS Y COSAS DE EVENTOS'''

data['Evento'] = data['Evento'].str.lower().str.strip()
data['Categoria'] = data['Categoria'].str.lower().str.strip()
data['Accion'] = data['Accion'].str.lower().str.strip()

## Estadisticas 

In [16]:
data.groupby(['Evento','Categoria'])['Jugada'].nunique().to_frame('Cantidad')

Cantidad
Evento    Categoria                 
a favor   ataque directo           1
          pelota parada            4
          transicion               2
en contra ataque combinado         1
          ataque directo           1
          pelota parada            2

### Armo las coordenadas

In [17]:
data['Zona inicial'] = data['Zona inicial'].replace('2A',2.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('2B',2.2,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8A',8.1,regex=True)
data['Zona inicial'] = data['Zona inicial'].replace('8B',8.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2A',2.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('2B',2.2,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8A',8.1,regex=True)
data['Zona Final'] = data['Zona Final'].replace('8B',8.2,regex=True)

In [18]:
data['Zona inicial'].value_counts()

8.2    12
2.2     6
2.1     4
5.0     4
8.1     4
4.0     4
3.0     2
6.0     2
9.0     1
1.0     1
7.0     1
Name: Zona inicial, dtype: int64

In [19]:
data['Zona Final'].value_counts()

5.0    9
2.1    6
8.2    6
8.1    4
6.0    4
4.0    4
1.0    2
7.0    2
2.2    2
9.0    1
3.0    1
Name: Zona Final, dtype: int64

In [20]:
map_zona_coord_x = {  
1:45,
2.1:70,
2.2:30,
3:45,
4:136,
5:136,
6:136,
7:227,
8.1:255,
8.2:207,
9:227
}

In [21]:
map_zona_coord_y = {  
1:135,
2.1:81,
2.2:81,
3:27,
4:135,
5:81,
6:27,
7:135,
8.1:81,
8.2:81,
9:81
}

In [22]:
data['coord.final.num'] = np.where(data['Zona Final'] == 'volver',None, data['Zona Final']).astype('float')


In [23]:
data['Zona inicial x'] = data['Zona inicial'].map(map_zona_coord_x)
data['Zona Final x'] = data['coord.final.num'].map(map_zona_coord_x)
data['Zona inicial y'] = data['Zona inicial'].map(map_zona_coord_y)
data['Zona Final y'] = data['coord.final.num'].map(map_zona_coord_y)
data.iloc[:,-5:]

,coord.final.num,Zona inicial x,Zona Final x,Zona inicial y,Zona Final y
0,2.1,70,70,81,81
1,2.1,136,70,81,81
2,5.0,255,136,81,81
3,8.1,207,255,81,81
4,8.2,207,207,81,81
5,5.0,136,136,81,81
6,5.0,207,136,81,81
7,8.2,255,207,81,81
8,8.1,207,255,81,81
9,1.0,70,45,81,135


In [24]:
# pasar en ambos formatos. Crear las dos cols originales q sean strings

data['coord.inicial'] = round(data['Zona inicial x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona inicial y'],0).astype('str').str.split('.').str[0]
data['coord.final'] = round(data['Zona Final x'],0).astype('str').str.split('.').str[0]+','+round(data['Zona Final y'],0).astype('str').str.split('.').str[0]


In [25]:
data['coord.inicial'] = np.where(data['coord.inicial']=='nan,nan', None, data['coord.inicial'])
data['coord.final'] = np.where(data['coord.final']=='nan,nan', None, data['coord.final'])

In [26]:
data[['coord.inicial','coord.final']]

,coord.inicial,coord.final
0,"70,81","70,81"
1,"136,81","70,81"
2,"255,81","136,81"
3,"207,81","255,81"
4,"207,81","207,81"
5,"136,81","136,81"
6,"207,81","136,81"
7,"255,81","207,81"
8,"207,81","255,81"
9,"70,81","45,135"


In [27]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41 entries, 0 to 40
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   Fecha                41 non-null     datetime64[ns]
 1   etiquetado           41 non-null     object        
 2   name                 41 non-null     object        
 3   Session Start        41 non-null     object        
 4   Session End          41 non-null     object        
 5   Evento               41 non-null     object        
 6   Tag Notes            0 non-null      float64       
 7   Tag Start            41 non-null     object        
 8   Tag End              41 non-null     object        
 9   Tag Duration (secs)  41 non-null     float64       
 10  Categoria            41 non-null     object        
 11  Zona inicial         41 non-null     float64       
 12  Zona Final           41 non-null     float64       
 13  Accion               41 non-null     

In [ ]:
data['Temporada'] = 'Temp 02-2022'
data['partido'] = 'Newells'

In [ ]:
data.to_excel("../Datos generados/reserva02-22-fecha5_newells_situaciones.xlsx",index=False)